In [5]:
from ortools.linear_solver import pywraplp

with open("add.txt", "r") as f:
    text = f.read()
solver = pywraplp.Solver.CreateSolver('GLOP')
arg = []
for i in text:
    if i != ' ' and i != '\t':
        arg.append(int(i))

arg[3] = arg[3] - (arg[3]*2) # variable creating
x1 = solver.NumVar(0,9999,'x1')
x2 = solver.NumVar(0,9999,'x2')
x3 = solver.NumVar(0,9999,'x3')
x4 = solver.NumVar(0,9999,'x4')
x5 = solver.NumVar(0,9999,'x5')

ct1 = solver.Constraint(0, arg[4], 'ct1')# constraints creating
ct1.SetCoefficient(x1, 2)
ct1.SetCoefficient(x2, -3)
ct1.SetCoefficient(x3, 4)
ct1.SetCoefficient(x4, -1)
ct1.SetCoefficient(x5, 2)

ct2 = solver.Constraint(0, 16, 'ct2')
ct2.SetCoefficient(x1, 2)
ct2.SetCoefficient(x2, 1)
ct2.SetCoefficient(x3, 1)
ct2.SetCoefficient(x4, 1)

ct3 = solver.Constraint(0, arg[5], 'ct3')
ct3.SetCoefficient(x1, arg[7])
ct3.SetCoefficient(x3, -1)
ct3.SetCoefficient(x4, 2)

ct4 = solver.Constraint(0, arg[6], 'ct4')
ct3.SetCoefficient(x2, -1)
ct4.SetCoefficient(x5, -1)

ct5 = solver.Constraint(0, 8, 'ct5')
ct5.SetCoefficient(x1, 1)
ct5.SetCoefficient(x2, 1)

ct6 = solver.Constraint(0, 3, 'ct6')
ct6.SetCoefficient(x5, 1)

objective = solver.Objective()# Objective function creating
objective.SetCoefficient(x1, arg[0])
objective.SetCoefficient(x2, arg[1])
objective.SetCoefficient(x3, arg[2]) # -
objective.SetCoefficient(x4, arg[3])
objective.SetCoefficient(x5, 3)
objective.SetMaximization()
solver.Solve()# solving and showing results

print(x1.solution_value()*arg[0]+x2.solution_value()*arg[1]+x3.solution_value()*arg[2]+x4.solution_value()*arg[3]+x5.solution_value())
print(x1.solution_value())
print(x2.solution_value())
print(x3.solution_value())
print(x4.solution_value())
print(x5.solution_value())

62.068965517241374
2.1379310344827585
5.862068965517242
4.827586206896552
0.0
0.0
